# Model 1

## Model: Classification trees and random forests <br>
### Considerations:
- The original data set was filtered to contemplate only two possible outcomes in the column `koi_disposition`: CONFIRMED and FALSE POSITIVE
- This model uses all the columns from the original data set
- The column `koi_tce_plnt_num` was treated as a categorical feature with eight possible outcomes, all of which were included in the model as dummy features. <br>
 
### Key takeaways:
- Good fit results were obtained both on the training and the testing sets, even before fitting with grid search
- The `koi_fpflag_co` feature has the biggest importance in the classification model.
- Outcomes from the `koi_tce_plnt_num` are the least relevant in the model. This feature could be dropped from the dataset in future iterations to achieve computing efficiency.
- Random forest best fit at: {'bootstrap': True, 'max_depth': 90, 'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 300}   best_score_: 0.9738562091503268

### Resources:
- Koehrsen, Will. Hyperparameter Tuning the Random Forest in Python (2018). Link: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74


In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\javier\anaconda3\envs\pythonadv\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [13]:
df = pd.read_csv("resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [14]:
# Filter candidate features to work with confirmed and false positives
df = df[df["koi_disposition"] != "CANDIDATE"]
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [27]:
# Get dummy values from categorical variables
data_binary_encoded = pd.get_dummies(df, columns=["koi_disposition", "koi_tce_plnt_num"])
data_binary_encoded = data_binary_encoded.drop(columns=["koi_disposition_FALSE POSITIVE"])
data_binary_encoded.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_kepmag,koi_disposition_CONFIRMED,koi_tce_plnt_num_1,koi_tce_plnt_num_2,koi_tce_plnt_num_3,koi_tce_plnt_num_4,koi_tce_plnt_num_5,koi_tce_plnt_num_6,koi_tce_plnt_num_7,koi_tce_plnt_num_8
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,15.347,1,0,1,0,0,0,0,0,0
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,15.436,0,1,0,0,0,0,0,0,0
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,15.597,0,1,0,0,0,0,0,0,0
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,15.509,1,1,0,0,0,0,0,0,0
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,15.714,1,0,1,0,0,0,0,0,0


# Create a Train Test Split

Use `koi_disposition` for the y values

In [38]:
y = data_binary_encoded["koi_disposition_CONFIRMED"]
X = data_binary_encoded.drop(columns="koi_disposition_CONFIRMED")

feature_names = X.columns

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)


In [30]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,dec,koi_kepmag,koi_tce_plnt_num_1,koi_tce_plnt_num_2,koi_tce_plnt_num_3,koi_tce_plnt_num_4,koi_tce_plnt_num_5,koi_tce_plnt_num_6,koi_tce_plnt_num_7,koi_tce_plnt_num_8
979,0,1,1,1,41.077297,1.985000e-04,-1.985000e-04,379.128480,0.003110,-0.003110,...,51.761822,14.907,1,0,0,0,0,0,0,0
6201,0,1,0,0,13.639480,1.970000e-06,-1.970000e-06,144.531439,0.000123,-0.000123,...,49.707901,15.852,1,0,0,0,0,0,0,0
5648,0,1,0,0,14.854416,7.890000e-07,-7.890000e-07,136.559783,0.000044,-0.000044,...,43.884331,14.558,1,0,0,0,0,0,0,0
229,0,0,0,0,10.681695,5.170000e-05,-5.170000e-05,131.550820,0.003720,-0.003720,...,46.097389,15.065,0,0,0,1,0,0,0,0
1302,0,0,0,0,12.509458,7.470000e-05,-7.470000e-05,136.992170,0.004600,-0.004600,...,38.735691,13.860,0,1,0,0,0,0,0,0


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [31]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [43]:
# Import dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

# Fit the model to scaled data and get training and testing scores
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_scaled, y_train)

print("Decision Tree Classifier")
print(f"Training Data Score: {clf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {clf.score(X_test_scaled, y_test)}")

Decision Tree Classifier
Training Data Score: 1.0
Testing Data Score: 0.9811463046757164


In [44]:
# Fit the model to scaled data and get training and testing scores
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

print("Random Forest Classifier")
print(f"Training Data Score: {rf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test)}")


Random Forest Classifier
Training Data Score: 1.0
Testing Data Score: 0.9803921568627451


In [39]:
# Determine feature importances
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.11686972756815829, 'koi_fpflag_co'),
 (0.07826268328793534, 'koi_steff_err2'),
 (0.0665720226436519, 'koi_steff_err1'),
 (0.0658812709924457, 'koi_fpflag_ss'),
 (0.058945510535093795, 'koi_prad'),
 (0.05358128396047659, 'koi_fpflag_nt'),
 (0.046961376953118264, 'koi_prad_err1'),
 (0.04322169353644323, 'koi_prad_err2'),
 (0.04185487969908515, 'koi_duration_err2'),
 (0.03788509430405332, 'koi_duration_err1'),
 (0.03377638465285948, 'koi_fpflag_ec'),
 (0.029223075790881563, 'koi_insol_err1'),
 (0.0234758406714217, 'koi_time0bk_err1'),
 (0.023451433463126985, 'koi_time0bk_err2'),
 (0.02216201989490781, 'koi_srad_err1'),
 (0.021972476638021852, 'koi_model_snr'),
 (0.021073069910774553, 'koi_period'),
 (0.02047048520448338, 'koi_impact'),
 (0.016661175005350144, 'koi_slogg_err2'),
 (0.016559764061560595, 'koi_depth'),
 (0.01508543490905149, 'koi_insol'),
 (0.014991974925116605, 'koi_teq'),
 (0.014154923468633123, 'koi_insol_err2'),
 (0.01382341813217273, 'koi_depth_err2'),
 (0.0129581209

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [47]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

rf = RandomForestClassifier()

In [50]:
# Train the model with GridSearch
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   55.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 16.7min finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [51]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'bootstrap': True, 'max_depth': 90, 'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 300}
0.9738562091503268


In [59]:
# Fit the hypertuned model to scaled data and get training and testing scores
rf_tuned = RandomForestClassifier(bootstrap= True, max_depth= 90, max_features= 3, min_samples_leaf= 4, min_samples_split= 8, n_estimators=300)
rf_tuned = rf.fit(X_train_scaled, y_train)

print("Random Forest Classifier")
print(f"Training Data Score: {rf_tuned.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf_tuned.score(X_test_scaled, y_test)}")

Random Forest Classifier
Training Data Score: 1.0
Testing Data Score: 0.9773755656108597


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)